# Enhance MagIC file for Shaat dike data

Two MagIC files are generated from project research:
- The set of MagIC files exported from Demag_GUI associated with the thermal demagnetization data
- The MagIC file exported from the IRM database associated with the rock magnetic experiments

These files need to be merged together into a single contribution as was done in:
https://github.com/Swanson-Hysell-Group/2021_ECMB/blob/main/data/rockmag_pmag_MagIC/combine_MagIC.ipynb

## Import Python packages

In [1]:
import pmagpy.ipmag as ipmag
import pmagpy.contribution_builder as cb
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import shutil

pmag_rmag_dir = '../data/pmag'

## Paleomagnetic data

The data that have been analyzed using demag_gui with MagIC files being saved out of demag_gui are imported below.

In [2]:
pmag_dir_path = '../data/pmag/demag'
pmag_contribution = cb.Contribution(pmag_dir_path)
pmag_measurements = pmag_contribution.tables['measurements'].df
pmag_specimens = pmag_contribution.tables['specimens'].df
pmag_samples = pmag_contribution.tables['samples'].df
pmag_sites = pmag_contribution.tables['sites'].df
pmag_locations = pmag_contribution.tables['locations'].df

-I- Using online data model
-I- Getting method codes from earthref.org
-I- Importing controlled vocabularies from https://earthref.org


In [3]:
pmag_specimens.head()

,specimen,experiments,sample,dir_alpha95,dir_comp,dir_dec,dir_inc,dir_n_comps,dir_n_measurements,dir_tilt_correction,dir_dang,dir_mad_free,meas_step_max,meas_step_min,meas_step_unit,analysts,instrument_codes,software_packages,int_corr,citations,method_codes,result_quality,volume,weight,location,site
specimen name,,,,,,,,,,,,,,,,,,,,,,,,,,
AynC-1,AynC-1,None,AynC-1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,This study,LP-NOMAG,None,NaN,NaN,Zalawt plain,AynC
AynC-1,AynC-1,AynC-1_LP-DIR-T,AynC-1,NaN,lt,206.4,68.7,2.0,8.0,-1.0,8.6,1.0,573.0,273.0,K,NaN,hargrave 2023-10-15 17:59:17,pmagpy-4.2.125: demag_gui,u,This study,LP-DIR-T:DE-BFL:DA-DIR,g,NaN,NaN,Zalawt plain,AynC
AynC-1,AynC-1,AynC-1_LP-DIR-T,AynC-1,NaN,mag,327.1,-65.8,2.0,12.0,-1.0,0.0,2.0,863.0,783.0,K,NaN,hargrave 2023-10-15 17:59:17,pmagpy-4.2.125: demag_gui,u,This study,LP-DIR-T:DE-BFL-A:DA-DIR,g,NaN,NaN,Zalawt plain,AynC
AynC-1,AynC-1,AynC-1_LP-DIR-T,AynC-1,NaN,lt,2.8,23.9,2.0,8.0,0.0,8.6,1.0,573.0,273.0,K,NaN,hargrave 2023-10-15 17:59:17,pmagpy-4.2.125: demag_gui,u,This study,LP-DIR-T:DE-BFL:DA-DIR-GEO,g,NaN,NaN,Zalawt plain,AynC
AynC-1,AynC-1,AynC-1_LP-DIR-T,AynC-1,NaN,mag,207.2,-25.0,2.0,12.0,0.0,0.0,2.0,863.0,783.0,K,NaN,hargrave 2023-10-15 17:59:17,pmagpy-4.2.125: demag_gui,u,This study,LP-DIR-T:DE-BFL-A:DA-DIR-GEO,g,NaN,NaN,Zalawt plain,AynC


### Paleomagnetic sites 

#### Enhance the site location precision and add site metadata

An issue with the CIT data format is that the precision on the site longitude/latitude is low given the ASCII file format. We therefore need to bring in more precise locations which are within the site_locations.csv file and add them to the site table.

Additionally, MagIC requires that sites have additional metadata including:
- geologic_classes
- geologic_types
- lithologies
- age (or age_high and age_low)
- age_unit
  
These metadata are within the site_locations.csv file are can be merged into the sites table in the code below.

In [4]:
# get more precise site locations
SD_site_locations = pd.read_csv('../data/field_data/site_locations.csv', sep=',', header=0, index_col='site')
# map site locations to site data
pmag_sites['lat'] = SD_site_locations['latitude'][pmag_sites['site']].tolist()
pmag_sites['lon'] = SD_site_locations['longitude'][pmag_sites['site']].tolist()
pmag_sites['geologic_classes'] = SD_site_locations['geologic_classes'][pmag_sites['site']].tolist()
pmag_sites['geologic_types'] = SD_site_locations['geologic_types'][pmag_sites['site']].tolist()
pmag_sites['lithologies'] = SD_site_locations['lithologies'][pmag_sites['site']].tolist()
pmag_sites['age_high'] = SD_site_locations['age_high'][pmag_sites['site']].tolist()
pmag_sites['age_low'] = SD_site_locations['age_low'][pmag_sites['site']].tolist()
pmag_sites['age_unit'] = SD_site_locations['age_unit'][pmag_sites['site']].tolist()

The sites that do not have a site mean calculated for them in Pmag_GUI do not have a citation generated. Here we fill in those `None` values with 'This study'

In [5]:
pmag_sites['citations'] = pmag_sites['citations'].fillna('This study')

All site locations were determined with GPS and that should be added to the method codes. The code below appends that to the existing method codes or puts that in as the single method codes if they are not yet other method codes.

In [6]:
pmag_sites['method_codes'] = pmag_sites['method_codes'].apply(lambda x: 'FS-LOC-GPS' if x is None else x + ':FS-LOC-GPS')

The criteria column in the sites is not used for this data set so will be dropped.

In [7]:
pmag_sites = pmag_sites.drop(columns=['criteria'])

#### Filter out tilt-corrected specimens and sites when not in Leger panel

We only have strong structural control in the Leger panel. We exported both geographic and tilt-corrected coordinates for every specimen and site out of Pmag_GUI, but should only report tilt-corrected data (dir_tilt_correction of 100) for the Leger panel. We can find the sites within the Leger panel and only keep pmag_specimens and pmag_sites with ['dir_tilt_correction'] of 100.

In [8]:
leger_sites = SD_site_locations[SD_site_locations['locale'].str.contains("Leger", na=False)].index
leger_sites

Index(['SD36', 'SD37', 'SD38', 'SD39', 'SD40', 'SD42', 'SD43', 'SD44', 'SD45',
       'SD46', 'SD47', 'SD48', 'SD49', 'SD50', 'SD51', 'SD52', 'SD53', 'SD54',
       'SD55', 'SD56', 'SD57', 'SD58', 'SD59', 'SD60', 'SD61', 'SD62', 'SD63',
       'SD64', 'SD65', 'SD66', 'SD67', 'SD68', 'SD69', 'SD70', 'SD71', 'SD72',
       'SD73', 'SD74', 'SD75', 'SD76', 'SD77', 'SD78', 'SD79', 'SD80', 'SD86',
       'SD87', 'SD88', 'SD89', 'SD90', 'SD91', 'SD92', 'SD93', 'SD94', 'SD95',
       'SD96', 'SD97', 'SD98', 'SD99', 'S100', 'S101', 'S102', 'S103', 'S104',
       'S105', 'S106', 'S107'],
      dtype='object', name='site')

We are going to filter the sites table so that it has geographic for all sites and tilt corrected for the Leger panel. We also need to include a few extra sites that don't have site means, but that have samples in the samples table.

In [9]:
extra_sites = ["SD76", "SD15", "S102", "SD98", "S105", "SD66", "SD73"]

In [10]:
mask = (
    (pmag_sites["dir_tilt_correction"] == 0)
    | (
        (pmag_sites["dir_tilt_correction"] == 100)
        & pmag_sites["site"].isin(leger_sites)
    )
    | pmag_sites["site"].isin(extra_sites)
)

pmag_sites = pmag_sites[mask]

pmag_specimens = pmag_specimens[
    (pmag_specimens["dir_tilt_correction"] == 0)
    | (
        (pmag_specimens["dir_tilt_correction"] == 100)
        & pmag_specimens["site"].isin(leger_sites)
    )
]

### Paleomagnetic specimens

All specimens were cut from their sample with a brass diamond saw. We add the method code 'SP-SAW' to indicate this.

We will also make sure all specimens refer to this study as the citation.

In [11]:
pmag_specimens['method_codes'] = pmag_specimens['method_codes'].apply(lambda x: 'SP-SAW' if x is None else x + ':SP-SAW')

In [12]:
pmag_specimens['citations'] = pmag_specimens['citations'].fillna('This study')

### Paleomagnetic locations 

We need to add necessary metadata to the locations table. The following information needs to be provided:
- location_type
- geologic_classes
- lithologies
- age *or age_low and age_high*
- age_unit

In [13]:
unique_geologic_classes = pmag_sites['geologic_classes'].dropna().unique()
location_geologic_classes = ": ".join(map(str, unique_geologic_classes))

unique_lithologies = pmag_sites['lithologies'].dropna().unique()
location_lithologies = ": ".join(map(str, unique_lithologies))

highest_age_high = pmag_sites['age_high'].max()
lowest_age_low = pmag_sites['age_low'].min()

In [14]:
pmag_locations['location_type'] = 'Region'
pmag_locations['geologic_classes'] = location_geologic_classes
pmag_locations['lithologies'] = location_lithologies
pmag_locations['age_low'] = lowest_age_low
pmag_locations['age_high'] = highest_age_high
pmag_locations['age_unit'] = 'Ma'
pmag_locations

,analysts,citations,lat_n,lat_s,location,lon_e,lon_w,location_type,geologic_classes,lithologies,age_low,age_high,age_unit
location name,,,,,,,,,,,,,
Zalawt plain,NaN,This study,17.1,17.1,Zalawt plain,54.9,54.9,Region,Sedimentary: Igneous: Metamorphic,Diamictite: Felsic Dike: Mafic Dike: Amphiboli...,660,815,Ma


### Paleomagnetic data enhanced table save

In [15]:
pmag_contribution.tables['measurements'].df = pmag_measurements
pmag_contribution.tables['specimens'].df = pmag_specimens
pmag_contribution.tables['samples'].df = pmag_samples
pmag_contribution.tables['sites'].df = pmag_sites
pmag_contribution.tables['locations'].df = pmag_locations

In [16]:
pmag_enhanced_dir = '../data/pmag/demag_enhanced'
pmag_contribution.tables['locations'].write_magic_file(dir_path=pmag_enhanced_dir,custom_name = "locations.txt")
pmag_contribution.tables['sites'].write_magic_file(dir_path=pmag_enhanced_dir,custom_name = "sites.txt")
pmag_contribution.tables['samples'].write_magic_file(dir_path=pmag_enhanced_dir,custom_name = "samples.txt")
pmag_contribution.tables['specimens'].write_magic_file(dir_path=pmag_enhanced_dir,custom_name = "specimens.txt")
pmag_contribution.tables['measurements'].write_magic_file(dir_path=pmag_enhanced_dir,custom_name = "measurements.txt")

-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/demag_enhanced/locations.txt
-I- 1 records written to locations file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/demag_enhanced/sites.txt
-I- 301 records written to sites file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/demag_enhanced/samples.txt
-I- 386 records written to samples file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/demag_enhanced/specimens.txt
-I- 738 records written to specimens file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/demag_enhanced/measurements.txt
-I- 9099 records written to measurements file


'/Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/demag_enhanced/measurements.txt'

## Rock magnetic data

The rock magnetic data are within a specimens table and a measurements table.

In [17]:
ipmag.unpack_magic('../data/pmag/rockmag/Oman_23_24.TXT', dir_path='../data/pmag/rockmag')

working on:  'specimens\r'
19  records written to file  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/rockmag/specimens.txt
  data put in  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/rockmag/specimens.txt
working on:  'measurements\r'
20326  records written to file  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/rockmag/measurements.txt
  data put in  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/rockmag/measurements.txt


True

In [18]:
rmag_dir_path = '../data/pmag/rockmag'
rmag_contribution = cb.Contribution(rmag_dir_path)
rmag_measurements = rmag_contribution.tables['measurements'].df
rmag_specimens = rmag_contribution.tables['specimens'].df
rmag_measurements.head(1)

,analysts,aniso_s,aniso_type,citations,contribution_id,derived_value,description,dir_csd,dir_dec,dir_inc,display_order,experiment,external_database_ids,files,hyst_charging_mode,hyst_loop,hyst_sweep_rate,instrument_codes,inversion_height,inversion_residuals,magn_b_111,magn_b_111_sigma,magn_b_x,magn_b_x_sigma,magn_b_xyz_sigma,magn_b_y,magn_b_y_sigma,magn_b_z,magn_b_z_sigma,magn_induction,magn_mass,magn_moment,magn_r2_det,magn_uncal,magn_volume,magn_x,magn_x_sigma,magn_xyz_sigma,magn_y,magn_y_sigma,magn_z,magn_z_sigma,meas_duration,meas_field_ac,meas_field_ac_phi,meas_field_ac_theta,meas_field_dc,meas_field_dc_phi,meas_field_dc_theta,meas_freq,meas_n_orient,meas_orient_phi,meas_orient_theta,meas_pos_x,meas_pos_y,meas_pos_z,meas_temp,meas_temp_change,measurement,method_codes,quality,result_type,row_id,sequence,software_packages,specimen,standard,susc_chi_mass,susc_chi_qdr_mass,susc_chi_qdr_volume,susc_chi_volume,timestamp,treat_ac_field,treat_ac_field_dc_off,treat_ac_field_dc_on,treat_ac_field_decay_rate,treat_dc_field,treat_dc_field_ac_off,treat_dc_field_ac_on,treat_dc_field_decay_rate,treat_dc_field_phi,treat_dc_field_theta,treat_mw_integral,treat_mw_power,treat_mw_step,treat_mw_time,treat_step_num,treat_temp,treat_temp_dc_off,treat_temp_dc_on,treat_temp_decay_rate
measurement name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
IRM-VSM4-LP-HYS-241942-1,NaN,NaN,NaN,This study,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IRM-VSM4-LP-HYS-241942,NaN,NaN,NaN,NaN,NaN,IRM-VSM4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294.0,NaN,IRM-VSM4-LP-HYS-241942-1,LP-HYS,g,NaN,NaN,NaN,NaN,SD38-3,u,NaN,NaN,NaN,NaN,2024:10:25:10:33:27.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The specimens prepared from rock magnetism should have been given unique names relative to the samples. We will append an 'r' to the sample name to indicate that it was the chip that was prepared for rock magnetic experiments.

In [19]:
rmag_specimens['specimen'] = rmag_specimens['specimen'].apply(lambda x: x + 'r' if isinstance(x, str) else x)
rmag_measurements['specimen'] = rmag_measurements['specimen'].apply(lambda x: x + 'r' if isinstance(x, str) else x)

In [20]:
rmag_measurements['timestamp'] = pd.to_datetime(
    rmag_measurements['timestamp'],
    format='%Y:%m:%d:%H:%M:%S.%f',
    errors='raise'
)
rmag_measurements.head(1)

,analysts,aniso_s,aniso_type,citations,contribution_id,derived_value,description,dir_csd,dir_dec,dir_inc,display_order,experiment,external_database_ids,files,hyst_charging_mode,hyst_loop,hyst_sweep_rate,instrument_codes,inversion_height,inversion_residuals,magn_b_111,magn_b_111_sigma,magn_b_x,magn_b_x_sigma,magn_b_xyz_sigma,magn_b_y,magn_b_y_sigma,magn_b_z,magn_b_z_sigma,magn_induction,magn_mass,magn_moment,magn_r2_det,magn_uncal,magn_volume,magn_x,magn_x_sigma,magn_xyz_sigma,magn_y,magn_y_sigma,magn_z,magn_z_sigma,meas_duration,meas_field_ac,meas_field_ac_phi,meas_field_ac_theta,meas_field_dc,meas_field_dc_phi,meas_field_dc_theta,meas_freq,meas_n_orient,meas_orient_phi,meas_orient_theta,meas_pos_x,meas_pos_y,meas_pos_z,meas_temp,meas_temp_change,measurement,method_codes,quality,result_type,row_id,sequence,software_packages,specimen,standard,susc_chi_mass,susc_chi_qdr_mass,susc_chi_qdr_volume,susc_chi_volume,timestamp,treat_ac_field,treat_ac_field_dc_off,treat_ac_field_dc_on,treat_ac_field_decay_rate,treat_dc_field,treat_dc_field_ac_off,treat_dc_field_ac_on,treat_dc_field_decay_rate,treat_dc_field_phi,treat_dc_field_theta,treat_mw_integral,treat_mw_power,treat_mw_step,treat_mw_time,treat_step_num,treat_temp,treat_temp_dc_off,treat_temp_dc_on,treat_temp_decay_rate
measurement name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
IRM-VSM4-LP-HYS-241942-1,NaN,NaN,NaN,This study,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IRM-VSM4-LP-HYS-241942,NaN,NaN,NaN,NaN,NaN,IRM-VSM4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294.0,NaN,IRM-VSM4-LP-HYS-241942-1,LP-HYS,g,NaN,NaN,NaN,NaN,SD38-3r,u,NaN,NaN,NaN,NaN,2024-10-25 10:33:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
rmag_contribution.tables['measurements'].df = rmag_measurements
rmag_contribution.tables['specimens'].df = rmag_specimens

rmag_enhanced_dir = '../data/pmag/rockmag_enhanced'
rmag_contribution.tables['specimens'].write_magic_file(dir_path=rmag_enhanced_dir,custom_name = "specimens.txt")
rmag_contribution.tables['measurements'].write_magic_file(dir_path=rmag_enhanced_dir,custom_name = "measurements.txt")

-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/rockmag_enhanced/specimens.txt
-I- 19 records written to specimens file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/rockmag_enhanced/measurements.txt
-I- 20326 records written to measurements file


'/Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/rockmag_enhanced/measurements.txt'

## Merge rock magnetic and paleomagnetic MagIC tables

The specimens and measurements tables can be combined.

In [22]:
ipmag.combine_magic([rmag_enhanced_dir + '/specimens.txt',
                     pmag_enhanced_dir + '/specimens.txt'], 
                    outfile = pmag_rmag_dir + '/specimens.txt',
                    magic_table='specimens')

ipmag.combine_magic([rmag_enhanced_dir + '/measurements.txt',
                     pmag_enhanced_dir + '/measurements.txt'], 
                    outfile = pmag_rmag_dir + '/measurements.txt',
                    magic_table='measurements')

-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/specimens.txt
-I- 757 records written to specimens file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/measurements.txt
-I- 29425 records written to measurements file


'/Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/measurements.txt'

The samples, sites, and locations tables will be the ones from the pmag data as they have not been updated in conjunction with the rock magnetic data.

In [23]:
shutil.copy(pmag_enhanced_dir + '/locations.txt', pmag_rmag_dir + '/locations.txt')
shutil.copy(pmag_enhanced_dir + '/sites.txt', pmag_rmag_dir + '/sites.txt')
shutil.copy(pmag_enhanced_dir + '/samples.txt', pmag_rmag_dir + '/samples.txt')

'../data/pmag/samples.txt'

A new MagIC contribution can be made from the combined files

In [24]:
ipmag.upload_magic(dir_path=pmag_rmag_dir, input_dir_path=pmag_rmag_dir)

-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/locations.txt file successfully read in
1  records written to file  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/locations.txt written to  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/samples.txt file successfully read in
386  records written to file  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/samples.txt written to  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/specimens.txt file successfully read in
757  records written to file  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data

('/Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/Zalawt-plain_22.Apr.2025_1.txt',
 {'status': False, 'validation': [], 'warnings': 'Status code 500'},
 None,
 None)